<a href="https://colab.research.google.com/github/Hamlet-be/20240422_Gastcollege/blob/main/Excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's Practice

The purpose of this hands-on exercise is to provide a practical introduction into spatial feature engineering, a process of developing additional information from raw data using geographic knowledge.

Before starting we have to install and set up the environment.
Goolge Colab runs in a cloud virtual machine that uses [Ubuntu](https://ubuntu.com/). This means that it can process bash commands in addition to python logic. Any command with a '!' in front of it will be treated as if it were entered on the command line. This means that the [ArcGIS API for python](https://developers.arcgis.com/python/) can be installed with the [pip](https://pip.pypa.io/en/stable/cli/) command.

## Set-up environment

In [ ]:
# check installed packages
!pip list

In [1]:
# install arcgis python (can take )
!pip install arcgis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pylerc-4.0-py3-none-any.whl
  Using cached ujson-5.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (53 kB)
  Using cached jupyterlab-4.1.6-py3-none-any.whl (11.4 MB)
  Using cached geomet-1.1.0-py3-none-any.whl (31 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached pyspnego-0.10.2-py3-none-any.whl (129 kB)
  Using cached requests_kerberos-0.14.0-py2.py3-none-any.whl (11 kB)
  Using cached requests_gssapi-1.3.0-py3-none-any.whl (12 kB)
  Using cached truststore-0.8.0-py3-none-any.whl (16 kB)
  Using cached async_lru-2.0.4-py3-none-any.whl (6.1 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached ipykernel-6.29.4-py3-none-any.whl (117 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl (69 kB)
  Using cached jupyter_server-2.14.0-py3-none-any.whl (383 kB)
  Using cached jupyterlab_s

In [2]:
# install rtree package needed for spatial index
!pip install rtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.2/535.2 kB 9.0 MB/s eta 0:00:00


In [3]:
# Define an helper function to convert a ArcGIS SEDF (Spatially Eneabled DataFrame) into a GeoPandas DataFrame
def sdf2gdf(sdf):
  my_sdf = sdf.copy(deep=True)
  my_sdf['geom']= gpd.GeoSeries.from_wkt(my_sdf[my_sdf.spatial.name].geom.WKT)
  my_gdf = gpd.GeoDataFrame(my_sdf, geometry='geom', crs=f"EPSG:{my_sdf.spatial.sr.wkid}")
  return my_gdf

## Read different datasets

The datasets are already published as Feature Service on ArcGIS online (PaaS solution). They are public available and can be easily accessed with an anonymous user account.

In [4]:
# import packages
import pandas as pd
from arcgis import GeoAccessor, GeoSeriesAccessor
from arcgis import GIS
import geopandas as gpd
import numpy as np

In [5]:
from  ipyleaflet import Map , GeoData, basemaps, LayersControl, GeoJSON, WKTLayer
from ipywidgets import Layout

In [6]:
# # connect to ArcGIS Online as anonymous user
gis_AGOL = GIS()

### Inhabitants

Read inhabitants dataset into an ArcGIS Python Spatially Enabled DataFrame ([SEDF](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/))

In [7]:
# retrieve inhabitants item from arcgis online
items = gis_AGOL.content.search(query='UGent, Gastcollege, Demo, Inhabitants', item_type='Feature Service')

In [8]:
# check result
for item in items:
  print(item)

<Item title:"inhabitants_zoi_2023" type:Feature Layer Collection owner:wketsman_demo1>


In [9]:
inhabitants_FL  = items[0].layers[0]

In [10]:
# create a Spatially Enabled DataFrame object (sdf)
inhabitants_sdf=inhabitants_FL.query(where= ("1=1"), out_fields=['OBJECTID','CD_REFNIS','CD_SECTOR', 'TOTAL', 'TX_DESCR_NL', 'TX_DESCR_SECTOR_NLL'], out_sr='4326').sdf
inhabitants_sdf

,ObjectId,CD_REFNIS,CD_SECTOR,TOTAL,TX_DESCR_NL,TX_DESCR_SECTOR_NLL,SHAPE
0,1,44085,44080B012,201,Lievegem,STOKTEVIJVER,"{""rings"": [[[3.55569068089625, 51.126866959234..."
1,2,43018,43018A001,786,Zelzate,ZELZATE - GROTE MARKT,"{""rings"": [[[3.81388399055102, 51.197860369515..."
2,3,44085,44080B091,86,Lievegem,RONSELE - VERSPREIDE BEWONING,"{""rings"": [[[3.55881260690818, 51.140615790720..."
3,4,43018,43018A012,2008,Zelzate,ZELZATE - CENTRUM,"{""rings"": [[[3.82529155327598, 51.203379144195..."
4,5,43018,43018A020,1291,Zelzate,TUSSEN OUD EN NIEUW KANAAL,"{""rings"": [[[3.80499013122676, 51.210120545032..."
...,...,...,...,...,...,...,...
500,501,44085,44080A10-,531,Lievegem,BEKE,"{""rings"": [[[3.60968085262486, 51.122015796264..."
501,502,44085,44080A193,688,Lievegem,ZOMERGEM - VERSPR. BEW.- OOST,"{""rings"": [[[3.59662612908333, 51.139787302014..."
502,503,44085,44080A29-,170,Lievegem,TER DURMEN,"{""rings"": [[[3.57966237004959, 51.099954015661..."
503,504,44085,44080B00-,269,Lievegem,RONSELE-KERN,"{""rings"": [[[3.55569068089625, 51.126866959234..."


In [11]:
# create a spatial index
inhabitants_sdf.spatial.sindex()

### Primary schools

Read the most up to date primary school dataset source: [Gent Open Data Portaal](https://data.stad.gent/)

In [12]:
items = gis_AGOL.content.search(query='UGent, Gastcollege, Demo, Primary Schools', item_type='Feature Service')

In [13]:
# check result
for item in items:
  print(item)

<Item title:"locaties_basisscholen_gent" type:Feature Layer Collection owner:wketsman_demo1>


In [14]:
primaryschool_FL  = items[0].layers[0]
#primaryschool_sdf = pd.DataFrame.spatial.from_layer(primaryschool_FL)
primaryschool_sdf=primaryschool_FL.query(where= ("1=1"), out_fields=['OBJECTID','adres','naam', 'globalid'], out_sr='4326').sdf
primaryschool_sdf

,ObjectId,adres,naam,globalid,SHAPE
0,1,"Kerkstraat 85, 9050 Gentbrugge",SBS Henri D'Haese,9bd78619-fe63-40b3-926e-05928b3bc972,"{""x"": 3.7544861350598397, ""y"": 51.044664423928..."
1,2,"Steenakker 4, 9000 Gent",SBS Het Prisma,88e7f1a0-daf9-4f4e-ba72-aedc53ab4507,"{""x"": 3.72126676105623, ""y"": 51.02483608963539..."
2,3,"Sint-Baafskouterstraat 129, 9040 Sint-Amandsberg",GO! BS De Wijze Boom,cfcb6929-1305-45c6-ba64-94c60244bf09,"{""x"": 3.75623427415401, ""y"": 51.0551574149722,..."
3,4,"Amand Casier de ter Bekenlaan 26, 9030 Mariakerke",GO! BS De Wijze Eik Mariakerke,cd872569-5d88-42f9-b4fe-cfa61a8c3a21,"{""x"": 3.67169902989239, ""y"": 51.07424365849989..."
4,5,"Karel Lodewijk Ledeganckstraat 4, 9000 Gent",GO! BS De Kleine Icarus Gent,34304ff5-dd75-4c4f-8b9c-2fbc5b10692a,"{""x"": 3.7239767565925006, ""y"": 51.036905478006..."
...,...,...,...,...,...
126,127,"Sint-Pietersaalststraat 78A, 9000 Gent",VLS KLIM,bf5d82bc-1d32-45ad-8568-0ba98db4f387,"{""x"": 3.71570240750937, ""y"": 51.0297003677123,..."
127,128,"Sint-Pietersaalststraat 86, 9000 Gent",VLS KLIM,cbb99c9e-5609-4729-ae24-6e4e2b2577f0,"{""x"": 3.7161274290421096, ""y"": 51.028937817281..."
128,129,"Sint-Pietersaalststraat 78A, 9000 Gent",VLS KLIM,8d4a44f4-9492-4dcd-a1ac-30acc39c7979,"{""x"": 3.71570240750937, ""y"": 51.0297003677123,..."
129,130,"Karel Lodewijk Dierickxstraat 28, 9000 Gent",VLSBO Macarius,89da934e-9a06-4369-b3ea-1df1b9af2da4,"{""x"": 3.7057364234838, ""y"": 51.052095412419085..."


In [15]:
# create a spatial index
primaryschool_sdf.spatial.sindex()

### Addresses

Read all the addresses.

In [16]:
items = gis_AGOL.content.search(query='UGent, Gastcollege, Demo, Address', item_type='Feature Service')
# check result
for item in items:
  print(item)

<Item title:"addresses_zoi" type:Feature Layer Collection owner:wketsman_demo1>


In [17]:
address_FL  = items[0].layers[0]
address_sdf=address_FL.query(where= ("status = 'current'"), out_fields=['OBJECTID','streetname_nl','housenumber', 'boxnumber', 'municipality_nl', 'status', 'niscode' ], out_sr='4326').sdf
address_sdf

,OBJECTID,streetname_nl,housenumber,boxnumber,municipality_nl,status,niscode,SHAPE
0,1,Lijsterstraat,13,<NA>,De Pinte,current,44012A011,"{""x"": 3.6414303390000695, ""y"": 50.989442812999..."
1,2,Boelenaar,4,<NA>,Gent,current,44021K512,"{""x"": 3.6560795150000396, ""y"": 51.052966425000..."
2,3,Ossenstraat,86,<NA>,Gent,current,44021A10-,"{""x"": 3.73947261400002, ""y"": 51.05066367799999..."
3,4,Gerststraat,12,<NA>,Gent,current,44021A040,"{""x"": 3.71752585500008, ""y"": 51.061862649, ""sp..."
4,5,Wolterslaan,127,<NA>,Gent,current,44021E042,"{""x"": 3.7500798650000307, ""y"": 51.048560940000..."
...,...,...,...,...,...,...,...,...
282157,334424,Kortrijksepoortstraat,29,0001,Gent,current,44021A31-,"{""x"": 3.7222450770000304, ""y"": 51.045263299999..."
282158,334425,Kortrijksepoortstraat,29,0101,Gent,current,44021A31-,"{""x"": 3.72224508000005, ""y"": 51.0452631200001,..."
282159,334426,Kortrijksepoortstraat,29,0201,Gent,current,44021A31-,"{""x"": 3.7222450770000304, ""y"": 51.045263299999..."
282160,334427,Kortrijksepoortstraat,29,0301,Gent,current,44021A31-,"{""x"": 3.72224508000005, ""y"": 51.0452631200001,..."


In [ ]:
# create an rtree spatial index
address_sdf.spatial.sindex(stype='rtree', reset=True)

## Spatial Feature Engineering
For a cluster analysis the data scientist needs an extra feature that indicates the number of people that are living in a 5 min walking distance for each school in Gent.

### Calculate service areas with [ArcGIS PaaS](https://developers.arcgis.com) solution

In [18]:
# Log into ArcGIS PaaS to calculate serviceareas.
# Create first an api key on the developers.arcgis.com platform and replace xxx with your api-key
PaaS_api_key = 'xxx'
gis_PaaS = GIS(api_key= PaaS_api_key) # connect to ArcGIS Online as anonymous user


In [19]:
# import the necessary package form the ArcGIS Python API to calculate the service areas
import arcgis.network as network
from arcgis.features import FeatureLayer, Feature, FeatureSet

In [22]:
# Add technical 'name' field used by the network module and give it a unique value
primaryschool_sdf['Name'] = primaryschool_sdf.globalid
# Convert the sdf to and arcGIS Featureset (input for generate_service_areas function)
primaryschool_FS = primaryschool_sdf.spatial.to_featureset()
primaryschool_FS.df # show content

,ObjectId,adres,naam,globalid,SHAPE,Name
0,1,"Kerkstraat 85, 9050 Gentbrugge",SBS Henri D'Haese,9bd78619-fe63-40b3-926e-05928b3bc972,"{""x"": 3.7544861350598397, ""y"": 51.044664423928...",9bd78619-fe63-40b3-926e-05928b3bc972
1,2,"Steenakker 4, 9000 Gent",SBS Het Prisma,88e7f1a0-daf9-4f4e-ba72-aedc53ab4507,"{""x"": 3.72126676105623, ""y"": 51.02483608963539...",88e7f1a0-daf9-4f4e-ba72-aedc53ab4507
2,3,"Sint-Baafskouterstraat 129, 9040 Sint-Amandsberg",GO! BS De Wijze Boom,cfcb6929-1305-45c6-ba64-94c60244bf09,"{""x"": 3.75623427415401, ""y"": 51.0551574149722,...",cfcb6929-1305-45c6-ba64-94c60244bf09
3,4,"Amand Casier de ter Bekenlaan 26, 9030 Mariakerke",GO! BS De Wijze Eik Mariakerke,cd872569-5d88-42f9-b4fe-cfa61a8c3a21,"{""x"": 3.67169902989239, ""y"": 51.07424365849989...",cd872569-5d88-42f9-b4fe-cfa61a8c3a21
4,5,"Karel Lodewijk Ledeganckstraat 4, 9000 Gent",GO! BS De Kleine Icarus Gent,34304ff5-dd75-4c4f-8b9c-2fbc5b10692a,"{""x"": 3.7239767565925006, ""y"": 51.036905478006...",34304ff5-dd75-4c4f-8b9c-2fbc5b10692a
...,...,...,...,...,...,...
126,127,"Sint-Pietersaalststraat 78A, 9000 Gent",VLS KLIM,bf5d82bc-1d32-45ad-8568-0ba98db4f387,"{""x"": 3.71570240750937, ""y"": 51.0297003677123,...",bf5d82bc-1d32-45ad-8568-0ba98db4f387
127,128,"Sint-Pietersaalststraat 86, 9000 Gent",VLS KLIM,cbb99c9e-5609-4729-ae24-6e4e2b2577f0,"{""x"": 3.7161274290421096, ""y"": 51.028937817281...",cbb99c9e-5609-4729-ae24-6e4e2b2577f0
128,129,"Sint-Pietersaalststraat 78A, 9000 Gent",VLS KLIM,8d4a44f4-9492-4dcd-a1ac-30acc39c7979,"{""x"": 3.71570240750937, ""y"": 51.0297003677123,...",8d4a44f4-9492-4dcd-a1ac-30acc39c7979
129,130,"Karel Lodewijk Dierickxstraat 28, 9000 Gent",VLSBO Macarius,89da934e-9a06-4369-b3ea-1df1b9af2da4,"{""x"": 3.7057364234838, ""y"": 51.052095412419085...",89da934e-9a06-4369-b3ea-1df1b9af2da4


In [23]:
# Calculate all the service areas. Pay attention on the gis parameter. This should be the PaaS environment and not the ArcGIS Online environment
result = network.analysis.generate_service_areas(facilities=primaryschool_FS,
                                                 break_values= '5',
                                                 break_units="Minutes",
                                                 travel_direction='Towards Facility',
                                                 travel_mode='Walking Time',
                                                 polygon_overlap_type='Disks',
                                                 impedance='Walk Time',
                                                 gis=gis_PaaS)

In [24]:
# Read the result into a sdf
serviceareas_sdf = result.service_areas.df
serviceareas_sdf

,ObjectID,Name,FromBreak,ToBreak,FacilityOID,FacilityID,Name_1,adres,naam,globalid,Breaks,AdditionalTime,AdditionalDistance,Shape_Length,Shape_Area,SHAPE
0,1,9bd78619-fe63-40b3-926e-05928b3bc972 : 0 - 5,0.0,5.0,1,1,9bd78619-fe63-40b3-926e-05928b3bc972,"Kerkstraat 85, 9050 Gentbrugge",SBS Henri D'Haese,9bd78619-fe63-40b3-926e-05928b3bc972,<NA>,0.0,0.0,0.031953,0.000049,"{""rings"": [[[3.7547378150000554, 51.0486193890..."
1,2,88e7f1a0-daf9-4f4e-ba72-aedc53ab4507 : 0 - 5,0.0,5.0,2,2,88e7f1a0-daf9-4f4e-ba72-aedc53ab4507,"Steenakker 4, 9000 Gent",SBS Het Prisma,88e7f1a0-daf9-4f4e-ba72-aedc53ab4507,<NA>,0.0,0.0,0.032303,0.000042,"{""rings"": [[[3.7219494570000506, 51.0288565430..."
2,3,cfcb6929-1305-45c6-ba64-94c60244bf09 : 0 - 5,0.0,5.0,3,3,cfcb6929-1305-45c6-ba64-94c60244bf09,"Sint-Baafskouterstraat 129, 9040 Sint-Amandsberg",GO! BS De Wijze Boom,cfcb6929-1305-45c6-ba64-94c60244bf09,<NA>,0.0,0.0,0.033024,0.000045,"{""rings"": [[[3.756534438000074, 51.05872539000..."
3,4,cd872569-5d88-42f9-b4fe-cfa61a8c3a21 : 0 - 5,0.0,5.0,4,4,cd872569-5d88-42f9-b4fe-cfa61a8c3a21,"Amand Casier de ter Bekenlaan 26, 9030 Mariakerke",GO! BS De Wijze Eik Mariakerke,cd872569-5d88-42f9-b4fe-cfa61a8c3a21,<NA>,0.0,0.0,0.031382,0.000038,"{""rings"": [[[3.6696228310000265, 51.0769161910..."
4,5,34304ff5-dd75-4c4f-8b9c-2fbc5b10692a : 0 - 5,0.0,5.0,5,5,34304ff5-dd75-4c4f-8b9c-2fbc5b10692a,"Karel Lodewijk Ledeganckstraat 4, 9000 Gent",GO! BS De Kleine Icarus Gent,34304ff5-dd75-4c4f-8b9c-2fbc5b10692a,<NA>,0.0,0.0,0.030374,0.000048,"{""rings"": [[[3.7230723460000377, 51.0398608550..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,127,bf5d82bc-1d32-45ad-8568-0ba98db4f387 : 0 - 5,0.0,5.0,127,127,bf5d82bc-1d32-45ad-8568-0ba98db4f387,"Sint-Pietersaalststraat 78A, 9000 Gent",VLS KLIM,bf5d82bc-1d32-45ad-8568-0ba98db4f387,<NA>,0.0,0.0,0.035809,0.000046,"{""rings"": [[[3.714313812000057, 51.03357267700..."
127,128,cbb99c9e-5609-4729-ae24-6e4e2b2577f0 : 0 - 5,0.0,5.0,128,128,cbb99c9e-5609-4729-ae24-6e4e2b2577f0,"Sint-Pietersaalststraat 86, 9000 Gent",VLS KLIM,cbb99c9e-5609-4729-ae24-6e4e2b2577f0,<NA>,0.0,0.0,0.028131,0.00004,"{""rings"": [[[3.7179070570000476, 51.0333480990..."
128,129,8d4a44f4-9492-4dcd-a1ac-30acc39c7979 : 0 - 5,0.0,5.0,129,129,8d4a44f4-9492-4dcd-a1ac-30acc39c7979,"Sint-Pietersaalststraat 78A, 9000 Gent",VLS KLIM,8d4a44f4-9492-4dcd-a1ac-30acc39c7979,<NA>,0.0,0.0,0.035809,0.000046,"{""rings"": [[[3.714313812000057, 51.03357267700..."
129,130,89da934e-9a06-4369-b3ea-1df1b9af2da4 : 0 - 5,0.0,5.0,130,130,89da934e-9a06-4369-b3ea-1df1b9af2da4,"Karel Lodewijk Dierickxstraat 28, 9000 Gent",VLSBO Macarius,89da934e-9a06-4369-b3ea-1df1b9af2da4,<NA>,0.0,0.0,0.033094,0.000045,"{""rings"": [[[3.7030849230000626, 51.0553567230..."


### Review results on a map

In [25]:
# we use ipyleaflet: convert sdf into gpd DataFrame. ArcGIs Python Map Widget API is at the moment not compatible with google COLAB
serviceareas_gdf = sdf2gdf(serviceareas_sdf)
primaryschool_gdf=sdf2gdf(primaryschool_sdf)

In [28]:
m = Map(center=(51.1, 3.7), zoom=10, scroll_wheel_zoom=True, layout=Layout(width='70%', height='500px'), basemap=basemaps.Esri.WorldTopoMap)
geo_data_primaryschools = GeoData(geo_dataframe = primaryschool_gdf, name = 'Primary School')
m.add(geo_data_primaryschools)
geo_data_primaryschools_SA = GeoData(geo_dataframe = serviceareas_gdf, name = 'Primary School SA', hover_style={'fillColor': 'red' , 'fillOpacity': 0.2})
m.add(geo_data_primaryschools_SA)
control = LayersControl(position='topright')
m.add(control)
m

Map(center=[51.1, 3.7], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

### Calculate the number of address points per statistical sector

In [29]:
summary_nis_address_df=address_sdf.groupby(by='niscode').size().to_frame(name= 'CountOfAddress').reset_index()
summary_nis_address_df

,niscode,CountOfAddress
0,43018A001,790
1,43018A012,979
2,43018A020,721
3,43018A033,355
4,43018A044,1046
...,...,...
495,44080B012,101
496,44080B091,40
497,44080C000,241
498,44080C090,159


In [30]:
# Combine summary_nis_address_df with inhabitants_sdf and calculate the number of inhabitants per address
summary_nis_address_inhabitants = summary_nis_address_df.merge(inhabitants_sdf[['CD_SECTOR', 'TOTAL']], how='left', left_on='niscode', right_on='CD_SECTOR')
summary_nis_address_inhabitants['InhabitantsPerAddress']=summary_nis_address_inhabitants['TOTAL']/summary_nis_address_inhabitants['CountOfAddress']
summary_nis_address_inhabitants

,niscode,CountOfAddress,CD_SECTOR,TOTAL,InhabitantsPerAddress
0,43018A001,790,43018A001,786,0.994937
1,43018A012,979,43018A012,2008,2.051073
2,43018A020,721,43018A020,1291,1.790569
3,43018A033,355,43018A033,699,1.969014
4,43018A044,1046,43018A044,2199,2.102294
...,...,...,...,...,...
495,44080B012,101,44080B012,201,1.990099
496,44080B091,40,44080B091,86,2.15
497,44080C000,241,44080C000,454,1.883817
498,44080C090,159,44080C090,383,2.408805


### Add number of inhabitants to each address

In [31]:
address_inhabitants_sdf = address_sdf.merge(summary_nis_address_inhabitants, how='left', on='niscode')[['OBJECTID','streetname_nl', 'housenumber', 'boxnumber', 'municipality_nl', 'niscode', 'InhabitantsPerAddress', 'SHAPE']]

In [33]:
# create an rtree inex
#address_inhabitants_sdf.spatial.sindex(stype='rtree')

In [34]:
address_inhabitants_sdf

,OBJECTID,streetname_nl,housenumber,boxnumber,municipality_nl,niscode,InhabitantsPerAddress,SHAPE
0,1,Lijsterstraat,13,<NA>,De Pinte,44012A011,2.17364,"{""x"": 3.6414303390000695, ""y"": 50.989442812999..."
1,2,Boelenaar,4,<NA>,Gent,44021K512,2.0625,"{""x"": 3.6560795150000396, ""y"": 51.052966425000..."
2,3,Ossenstraat,86,<NA>,Gent,44021A10-,1.469512,"{""x"": 3.73947261400002, ""y"": 51.05066367799999..."
3,4,Gerststraat,12,<NA>,Gent,44021A040,1.287545,"{""x"": 3.71752585500008, ""y"": 51.061862649, ""sp..."
4,5,Wolterslaan,127,<NA>,Gent,44021E042,2.043624,"{""x"": 3.7500798650000307, ""y"": 51.048560940000..."
...,...,...,...,...,...,...,...,...
282157,334424,Kortrijksepoortstraat,29,0001,Gent,44021A31-,0.824249,"{""x"": 3.7222450770000304, ""y"": 51.045263299999..."
282158,334425,Kortrijksepoortstraat,29,0101,Gent,44021A31-,0.824249,"{""x"": 3.72224508000005, ""y"": 51.0452631200001,..."
282159,334426,Kortrijksepoortstraat,29,0201,Gent,44021A31-,0.824249,"{""x"": 3.7222450770000304, ""y"": 51.045263299999..."
282160,334427,Kortrijksepoortstraat,29,0301,Gent,44021A31-,0.824249,"{""x"": 3.72224508000005, ""y"": 51.0452631200001,..."


### Identify all the addresses in the serviceareas

In [ ]:
# Not enough available memory and CPU on COLAB to run following statement with the Spatially Enabled DataFrame
#serviceareas_adress_sdf = serviceareas_sdf.spatial.join(address_inhabitants_sdf, how='left')

In [38]:
# Convert sdf to gdf
address_inhabitants_gdf = sdf2gdf(address_inhabitants_sdf)

In [39]:
# execute spatial join
address_servicearea_df = serviceareas_gdf.sjoin(address_inhabitants_gdf)
address_servicearea_df

,ObjectID,Name,FromBreak,ToBreak,FacilityOID,FacilityID,Name_1,adres,naam,globalid,...,geom,index_right,OBJECTID,streetname_nl,housenumber,boxnumber,municipality_nl,niscode,InhabitantsPerAddress,SHAPE_right
0,1,9bd78619-fe63-40b3-926e-05928b3bc972 : 0 - 5,0.0,5.0,1,1,9bd78619-fe63-40b3-926e-05928b3bc972,"Kerkstraat 85, 9050 Gentbrugge",SBS Henri D'Haese,9bd78619-fe63-40b3-926e-05928b3bc972,...,"MULTIPOLYGON (((3.75474 51.04862, 3.75586 51.0...",99464,117701,Leo Tertzweillaan,30,<NA>,Gent,44021F001,2.074723,"{""x"": 3.7532518820000296, ""y"": 51.042074341, ""..."
0,1,9bd78619-fe63-40b3-926e-05928b3bc972 : 0 - 5,0.0,5.0,1,1,9bd78619-fe63-40b3-926e-05928b3bc972,"Kerkstraat 85, 9050 Gentbrugge",SBS Henri D'Haese,9bd78619-fe63-40b3-926e-05928b3bc972,...,"MULTIPOLYGON (((3.75474 51.04862, 3.75586 51.0...",124161,146940,Kliniekstraat,53,<NA>,Gent,44021F001,2.074723,"{""x"": 3.75483312400007, ""y"": 51.04139562299999..."
49,50,d8db1ebd-d6fc-449f-bea1-d81c85467da7 : 0 - 5,0.0,5.0,50,50,d8db1ebd-d6fc-449f-bea1-d81c85467da7,"Tweekapellenstraat 38, 9050 Gentbrugge",SBS Henri D'Haese,d8db1ebd-d6fc-449f-bea1-d81c85467da7,...,"MULTIPOLYGON (((3.75541 51.04705, 3.75609 51.0...",124161,146940,Kliniekstraat,53,<NA>,Gent,44021F001,2.074723,"{""x"": 3.75483312400007, ""y"": 51.04139562299999..."
0,1,9bd78619-fe63-40b3-926e-05928b3bc972 : 0 - 5,0.0,5.0,1,1,9bd78619-fe63-40b3-926e-05928b3bc972,"Kerkstraat 85, 9050 Gentbrugge",SBS Henri D'Haese,9bd78619-fe63-40b3-926e-05928b3bc972,...,"MULTIPOLYGON (((3.75474 51.04862, 3.75586 51.0...",11597,13768,Kliniekstraat,51,<NA>,Gent,44021F001,2.074723,"{""x"": 3.75484692400005, ""y"": 51.04143057400011..."
49,50,d8db1ebd-d6fc-449f-bea1-d81c85467da7 : 0 - 5,0.0,5.0,50,50,d8db1ebd-d6fc-449f-bea1-d81c85467da7,"Tweekapellenstraat 38, 9050 Gentbrugge",SBS Henri D'Haese,d8db1ebd-d6fc-449f-bea1-d81c85467da7,...,"MULTIPOLYGON (((3.75541 51.04705, 3.75609 51.0...",11597,13768,Kliniekstraat,51,<NA>,Gent,44021F001,2.074723,"{""x"": 3.75484692400005, ""y"": 51.04143057400011..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,131,81b340d4-ed82-4e6e-a106-3d8fe19f2952 : 0 - 5,0.0,5.0,131,131,81b340d4-ed82-4e6e-a106-3d8fe19f2952,"Ottergemsesteenweg 155, 9000 Gent",Vrije Basisschool,81b340d4-ed82-4e6e-a106-3d8fe19f2952,...,"MULTIPOLYGON (((3.72734 51.03762, 3.72914 51.0...",26846,31766,Burggravenlaan,334,<NA>,Gent,44021A33-,1.052539,"{""x"": 3.7325647180000505, ""y"": 51.032111255000..."
130,131,81b340d4-ed82-4e6e-a106-3d8fe19f2952 : 0 - 5,0.0,5.0,131,131,81b340d4-ed82-4e6e-a106-3d8fe19f2952,"Ottergemsesteenweg 155, 9000 Gent",Vrije Basisschool,81b340d4-ed82-4e6e-a106-3d8fe19f2952,...,"MULTIPOLYGON (((3.72734 51.03762, 3.72914 51.0...",107130,126725,Gustaaf Vanden Meerschestraat,3,<NA>,Gent,44021A33-,1.052539,"{""x"": 3.73286961000002, ""y"": 51.03324942000009..."
130,131,81b340d4-ed82-4e6e-a106-3d8fe19f2952 : 0 - 5,0.0,5.0,131,131,81b340d4-ed82-4e6e-a106-3d8fe19f2952,"Ottergemsesteenweg 155, 9000 Gent",Vrije Basisschool,81b340d4-ed82-4e6e-a106-3d8fe19f2952,...,"MULTIPOLYGON (((3.72734 51.03762, 3.72914 51.0...",13759,16282,Gustaaf Vanden Meerschestraat,2,<NA>,Gent,44021A33-,1.052539,"{""x"": 3.73281558500003, ""y"": 51.03330153600008..."
130,131,81b340d4-ed82-4e6e-a106-3d8fe19f2952 : 0 - 5,0.0,5.0,131,131,81b340d4-ed82-4e6e-a106-3d8fe19f2952,"Ottergemsesteenweg 155, 9000 Gent",Vrije Basisschool,81b340d4-ed82-4e6e-a106-3d8fe19f2952,...,"MULTIPOLYGON (((3.72734 51.03762, 3.72914 51.0...",87428,103462,Gustaaf Vanden Meerschestraat,1,<NA>,Gent,44021A33-,1.052539,"{""x"": 3.7326296640000596, ""y"": 51.033428811000..."


### Calculate number of inhabitants in service are

In [40]:
inhabitant_servicearea_df = address_servicearea_df.groupby('globalid')['InhabitantsPerAddress'].sum().to_frame(name='Inhabitants_5min').reset_index()
inhabitant_servicearea_df

,globalid,Inhabitants_5min
0,00be3e91-2e4f-42fe-bc42-0c692161f9e5,2483.421541
1,019475ec-8bcb-4938-a0a7-324b24e8ef58,3997.434862
2,038e9f0a-e903-42de-9ec7-5a6c15e508eb,3780.881786
3,09870779-379e-4839-9e7f-df68073d9874,2136.493401
4,0a67336a-cc60-4d7d-ac93-9711cf15dd87,1218.465157
...,...,...
126,ec1e90b0-427b-4adb-912e-6d9a52893eb0,251.446522
127,f3c83b58-5bb6-47cc-8ebc-81556c447a03,2821.094746
128,f9a5b200-cd0e-4a18-8c09-dabd6e166ced,2584.854134
129,fa5859a2-e1fb-46a4-8f7a-0881553beace,631.089036


### Add new calculated feature to initial dataset

In [41]:
primaryschool_gdf = sdf2gdf(primaryschool_sdf)
pd.merge(primaryschool_gdf,inhabitant_servicearea_df, on='globalid', how='left')

,ObjectId,adres,naam,globalid,SHAPE,Name,geom,Inhabitants_5min
0,1,"Kerkstraat 85, 9050 Gentbrugge",SBS Henri D'Haese,9bd78619-fe63-40b3-926e-05928b3bc972,"{""x"": 3.7544861350598397, ""y"": 51.044664423928...",9bd78619-fe63-40b3-926e-05928b3bc972,POINT (3.75449 51.04466),2443.354026
1,2,"Steenakker 4, 9000 Gent",SBS Het Prisma,88e7f1a0-daf9-4f4e-ba72-aedc53ab4507,"{""x"": 3.72126676105623, ""y"": 51.02483608963539...",88e7f1a0-daf9-4f4e-ba72-aedc53ab4507,POINT (3.72127 51.02484),1913.754361
2,3,"Sint-Baafskouterstraat 129, 9040 Sint-Amandsberg",GO! BS De Wijze Boom,cfcb6929-1305-45c6-ba64-94c60244bf09,"{""x"": 3.75623427415401, ""y"": 51.0551574149722,...",cfcb6929-1305-45c6-ba64-94c60244bf09,POINT (3.75623 51.05516),1677.893737
3,4,"Amand Casier de ter Bekenlaan 26, 9030 Mariakerke",GO! BS De Wijze Eik Mariakerke,cd872569-5d88-42f9-b4fe-cfa61a8c3a21,"{""x"": 3.67169902989239, ""y"": 51.07424365849989...",cd872569-5d88-42f9-b4fe-cfa61a8c3a21,POINT (3.67170 51.07424),419.7909
4,5,"Karel Lodewijk Ledeganckstraat 4, 9000 Gent",GO! BS De Kleine Icarus Gent,34304ff5-dd75-4c4f-8b9c-2fbc5b10692a,"{""x"": 3.7239767565925006, ""y"": 51.036905478006...",34304ff5-dd75-4c4f-8b9c-2fbc5b10692a,POINT (3.72398 51.03691),1515.894438
...,...,...,...,...,...,...,...,...
126,127,"Sint-Pietersaalststraat 78A, 9000 Gent",VLS KLIM,bf5d82bc-1d32-45ad-8568-0ba98db4f387,"{""x"": 3.71570240750937, ""y"": 51.0297003677123,...",bf5d82bc-1d32-45ad-8568-0ba98db4f387,POINT (3.71570 51.02970),1831.180808
127,128,"Sint-Pietersaalststraat 86, 9000 Gent",VLS KLIM,cbb99c9e-5609-4729-ae24-6e4e2b2577f0,"{""x"": 3.7161274290421096, ""y"": 51.028937817281...",cbb99c9e-5609-4729-ae24-6e4e2b2577f0,POINT (3.71613 51.02894),1490.592999
128,129,"Sint-Pietersaalststraat 78A, 9000 Gent",VLS KLIM,8d4a44f4-9492-4dcd-a1ac-30acc39c7979,"{""x"": 3.71570240750937, ""y"": 51.0297003677123,...",8d4a44f4-9492-4dcd-a1ac-30acc39c7979,POINT (3.71570 51.02970),1831.180808
129,130,"Karel Lodewijk Dierickxstraat 28, 9000 Gent",VLSBO Macarius,89da934e-9a06-4369-b3ea-1df1b9af2da4,"{""x"": 3.7057364234838, ""y"": 51.052095412419085...",89da934e-9a06-4369-b3ea-1df1b9af2da4,POINT (3.70574 51.05210),2610.824969
